<a href="https://colab.research.google.com/github/EdwarMontano/AITuringBackend/blob/main/Remote_Trainer_RecognizeMedications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table>
<tbody>
<tr>
<td><img width="100px" src="https://upload.wikimedia.org/wikipedia/commons/e/eb/Univalle.svg" alt="Univalle"> </td>
<td><img width="220px" src="https://i.ibb.co/6vdWxb4/PSI-LOGO.png" alt="PSI"></td>
</tr>
</tbody>
</table>

# Como Entrenar un modelo con de reconocimiento de medicamentos con  YOLOv8 con etiquetaas cuadro delimintadores orientados

---
[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/EdwarMontano/Trainer_RecognizeMedications)

## Consejo: Utiliza la aceleración por GPU

Si está ejecutando este cuaderno en Google Colab, vaya a `Editar` -> `Configuraciones de Notebook ` -> `Aceleración por Hardware `, configurarlo en `GPU`, y luego hacer clic en `Guardar`. Esto garantizará que tu cuaderno utilice una GPU, lo que acelerará significativamente los tiempos de entrenamiento del modelo.

## Pasos de este tutorial

En este tutorial, vamos a cubrir:

- Verificar que nuestro entorno de Colab tenga una GPU
- Instalar YOLOv8
- Preparar un conjunto de datos
- Entrenar un modelo YOLOv8 OBB
- Ejecutar la inferencia en nuestro modelo

¡Sin más preámbulos, comencemos!

## Antes de empezar

Asegurémonos de que tenemos acceso a la GPU. Para ello podemos usar el comando `nvidia-smi`. En caso de cualquier problema navegue a `Edit` -> `Notebook settings` -> `Hardware accelerator`, póngalo en `GPU`, y pulse `Save`.

In [1]:
!nvidia-smi

Tue Jun 11 03:40:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   61C    P8              13W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install YOLOv8

To install YOL0v8, run the following command:

In [3]:
!pip install ultralytics -q
!pip install roboflow --quiet
!pip install "comet_ml>=3.38.0" --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.6/780.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 9.2 MB/s eta 0:00:00
Reason for being yanked: deprecated, use 4.8.0.76
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 k

In [5]:
# import tensorflow as tf
import datetime, os

In [6]:
# Load Comet
import comet_ml
comet_ml.init(project_name="experiments-recognizemedications")

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


In [7]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.31 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Setup complete ✅ (12 CPUs, 53.0 GB RAM, 30.4/201.2 GB disk)


## Preparar un conjunto de datos personalizado

Construir un conjunto de datos personalizado puede ser un proceso tedioso. Puede tomar docenas o incluso cientos de horas recolectar imágenes, etiquetarlas y exportarlas en el formato adecuado. Afortunadamente, Roboflow hace que este proceso sea lo más sencillo y rápido posible. ¡Déjanos mostrarte cómo!


### Paso 1: Crear un proyecto

Antes de comenzar, necesitas crear una [cuenta](https://app.roboflow.com/login) en Roboflow. Una vez que lo hagas, puedes crear un nuevo proyecto en el [panel de control](https://app.roboflow.com/) de Roboflow. Elige "Detección de Objetos" como el tipo de proyecto.

<img src="https://github.com/EdwarMontano/Trainer_RecognizeMedications/assets/52388306/3afa0b1b-5629-491f-b145-9f7ba9a77325" alt="Detección de objetos seleccionada en el cuadro de diálogo Crear Proyecto de Roboflow" height="300" />

### Paso 2: Subir imágenes

A continuación, añade los datos a tu proyecto recién creado. Puedes hacerlo a través de la API o mediante nuestra [interfaz web](https://docs.roboflow.com/adding-data/object-detection).

Si arrastras y sueltas un directorio con un conjunto de datos en un formato compatible, el panel de control de Roboflow leerá automáticamente las imágenes y anotaciones juntos.

<img src="https://github.com/EdwarMontano/Trainer_RecognizeMedications/assets/52388306/40dd4aec-8756-4852-bcb9-cfc316bb1e3a" alt="Subiendo imágenes a Roboflow" height="300" />

### Paso 3: Etiquetar datos

Si solo tienes imágenes, puedes etiquetarlas con cuadros delimitadores orientados en [Roboflow Annotate](https://docs.roboflow.com/annotate).

**Para etiquetar un cuadro delimitador orientado, usa nuestra herramienta de anotación de polígonos.**

También puedes tomar un conjunto de datos de segmentación existente de tu espacio de trabajo o de Roboflow Universe y exportarlo como OBB.

<img src="https://github.com/EdwarMontano/Trainer_RecognizeMedications/assets/52388306/9b0e7c1e-b71f-4d71-ac3a-6d4b0261d8d3" alt="Anotar una imagen" height="300" />

### Paso 4: Generar una nueva versión del conjunto de datos

Ahora que tenemos nuestras imágenes y anotaciones añadidas, podemos generar una versión del conjunto de datos. Al generar una versión, puedes optar por añadir preprocesamiento y aumentos. Este paso es completamente opcional, sin embargo, puede permitirte mejorar significativamente la robustez de tu modelo.

<img src="https://github.com/EdwarMontano/Trainer_RecognizeMedications/assets/52388306/9a88edc9-6e0f-48c6-9f53-5683d846bb55" alt="Generar una versión del conjunto de datos" height="300" />

### Paso 5: Exportar conjunto de datos

Una vez que se genera la versión del conjunto de datos, podemos descargarla para usarla en el entrenamiento de un modelo.

![Generar una versión del conjunto de datos](https://media.roboflow.com/keypoint/export.png)

## Cargar Dataset

In [10]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets


import roboflow

roboflow.login()

rf = roboflow.Roboflow()

project = rf.workspace("uv-ipsi6").project("recognice-medications-boxes")
dataset = project.version(5).download("yolov8-obb")

import yaml

with open(f'{dataset.location}/data.yaml', 'r') as file:
    data = yaml.safe_load(file)

data['path'] = dataset.location

with open(f'{dataset.location}/data.yaml', 'w') as file:
    yaml.dump(data, file, sort_keys=False)

mkdir: cannot create directory ‘/content/datasets’: File exists
/content/datasets
visit https://app.roboflow.com/auth-cli to get your authentication token.
Paste the authentication token here: ··········
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Recognice-medications-boxes-5 in yolov8-obb:: 100%|██████████| 2724/2724 [00:00<00:00, 7839.52it/s]


In [12]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")

# Tune hyperparameters on data for 20 epochs
# model.tune(data="/content/datasets/Recognice-medications-boxes-5/data.yaml", epochs=20, iterations=10, optimizer="AdamW")

# 10/10 iterations complete ✅ (6425.85s)
# Results saved to runs/detect/tune3
# Best fitness=0.93773 observed at iteration 3
# Best fitness metrics are {'metrics/precision(B)': 0.99262, 'metrics/recall(B)': 0.93423, 'metrics/mAP50(B)': 0.95133, 'metrics/mAP50-95(B)': 0.93621, 'val/box_loss': 0.30765, 'val/cls_loss': 0.33356, 'val/dfl_loss': 0.77449, 'fitness': 0.93773}
# Best fitness model is runs/detect/train307
# Best fitness hyperparameters are printed below.

# lr0: 0.00825
# lrf: 0.01025
# momentum: 0.87414
# weight_decay: 0.00052
# warmup_epochs: 3.0
# warmup_momentum: 0.80581
# box: 7.07983
# cls: 0.456
# dfl: 1.36741
# hsv_h: 0.01342
# hsv_s: 0.6055
# hsv_v: 0.3556
# degrees: 0.0
# translate: 0.11443
# scale: 0.497
# shear: 0.0
# perspective: 0.0
# flipud: 0.0
# fliplr: 0.50865
# bgr: 0.0
# mosaic: 0.80471
# mixup: 0.0
# copy_paste: 0.0



100%|██████████| 6.23M/6.23M [00:00<00:00, 131MB/s]


In [32]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8m.pt')  # load a pretrained model (recommended for training)

# Train the model with 2 GPUs
# results = model.train(data='/content/datasets/Recognice-medications-boxes-5/data.yaml',batch=-1,patience=8, epochs=80, imgsz=640, device=0)
results = model.train(
    data="/content/datasets/Recognice-medications-boxes-5/data.yaml",
    epochs=100,
    patience=10,
    batch=-1,
    imgsz=640,
    device=0,
    lr0= 0.00825,
    lrf= 0.01025,
    momentum= 0.87414,
    weight_decay= 0.00052,
    warmup_epochs= 3.0,
    warmup_momentum= 0.80581,
    box= 7.07983,
    cls= 0.456,
    dfl= 1.36741,
    hsv_h= 0.01342,
    hsv_s= 0.6055,
    hsv_v= 0.3556,
    degrees= 0.0,
    translate= 0.11443,
    scale= 0.497,
    shear= 0.0,
    perspective= 0.0,
    flipud= 0.0,
    fliplr= 0.50865,
    bgr= 0.0,
    mosaic= 0.80471,
    mixup= 0.0,
    copy_paste= 0.0
)

Ultralytics YOLOv8.2.31 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/datasets/Recognice-medications-boxes-5/data.yaml, epochs=100, time=None, patience=10, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labe

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: tensorboard, tensorflow, keras.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content/datasets' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/edwarmontano/experiments-recognizemedications/878536cf9da1454fb48de3f7ae59397c



TensorBoard: Start with 'tensorboard --logdir runs/detect/train5', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅
AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA L4) 22.17G total, 2.38G reserved, 0.41G allocated, 19.38G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    25862110       79.09         2.586         26.17         22.83        (1, 3, 640, 640)                    list
    25862110       158.2         1.439         27.51         20.19        (2, 3, 640, 640)                    list
    25862110       316.4         2.020         27.84         24.02        (4, 3, 640, 640)                    list
    25862110       632.8         3.414         38.58         43.71        (8, 3, 640, 640)                    list
    25862110       

train: Scanning /content/datasets/Recognice-medications-boxes-5/train/labels.cache... 1185 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1185/1185 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

                   all        114        121      0.992      0.934      0.957      0.953
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 49, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



59 epochs completed in 0.413 hours.
Optimizer stripped from runs/detect/train5/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train5/weights/best.pt, 52.0MB

Validating runs/detect/train5/weights/best.pt...
Ultralytics YOLOv8.2.31 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 218 layers, 25845550 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.73it/s]


                   all        114        121      0.993      0.934      0.965      0.955
           ALKASELTZER         13         15      0.995      0.733      0.856      0.824
          ASPIRINA_100         11         11      0.995          1      0.995      0.974
             BUSCAPINA         10         10      0.994          1      0.995      0.995
           DOLEX_GRIPA         13         13          1      0.846      0.923      0.923
              GAVISCON         13         13       0.99      0.846      0.943      0.906
       IBUFLASH_MIGRAN         12         12      0.993          1      0.995      0.995
     IBUPROFENO_GENFAR         13         13      0.995          1      0.995      0.984
        LEVONORGESTREL         11         11       0.98          1      0.995      0.995
     NORAVER_GRIPA_DIA         11         11      0.992          1      0.995      0.995
              VITAC_MK         12         12          1      0.917      0.958      0.953
Speed: 0.2ms preproce

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : melodic_veranda_6659
COMET INFO:     url                   : https://www.comet.com/edwarmontano/experiments-recognizemedications/878536cf9da1454fb48de3f7ae59397c
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [215]                 : (39.819217681884766, 235.4358673095703)
COMET INFO:     lr/pg0 [119]               : (0.00023138888888888891, 0.0006933861258333333)
COMET INFO:     lr/pg1 [119]               : (0.00023138888888888891, 0.0006933861258333333)
COMET INFO:     lr/pg2 [119]               : (0.00023138888888888891, 0.0006933861258333333)
COMET INFO:     metrics/mAP50(B) [120]     : (0.60127, 0.9649547678571428)
COMET INFO:    

In [33]:
# Validate model

from ultralytics import YOLO

# Load a model
model = YOLO('/content/datasets/runs/detect/train5/weights/best.pt')  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

Ultralytics YOLOv8.2.31 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 218 layers, 25845550 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /content/datasets/Recognice-medications-boxes-5/valid/labels.cache... 114 images, 0 backgrounds, 0 corrupt: 100%|██████████| 114/114 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.54it/s]


                   all        114        121      0.993      0.934      0.963      0.954
           ALKASELTZER         13         15      0.995      0.733      0.838      0.818
          ASPIRINA_100         11         11      0.996          1      0.995      0.974
             BUSCAPINA         10         10      0.994          1      0.995      0.995
           DOLEX_GRIPA         13         13          1      0.846      0.923      0.923
              GAVISCON         13         13       0.99      0.846      0.943      0.906
       IBUFLASH_MIGRAN         12         12      0.993          1      0.995      0.995
     IBUPROFENO_GENFAR         13         13      0.995          1      0.995      0.984
        LEVONORGESTREL         11         11       0.98          1      0.995      0.995
     NORAVER_GRIPA_DIA         11         11      0.992          1      0.995      0.995
              VITAC_MK         12         12          1      0.917      0.958      0.953
Speed: 0.2ms preproce

array([    0.81762,     0.97396,       0.995,       0.923,     0.90625,       0.995,     0.98355,       0.995,       0.995,      0.9533])

In [34]:
#  Prediction using trained model
import os
from ultralytics import YOLO

image_dir = '/content/datasets/Recognice-medications-boxes-5/test/images'

# Check if the directory exists
if os.path.exists(image_dir):
    # Run inference if the directory exists
    model.predict(image_dir, save=True, imgsz=640, conf=0.2)
else:
    print(f"Error: The specified path '{image_dir}' does not exist.")

# Load a pretrained YOLOv8n model
model = YOLO('/content/datasets/runs/detect/train5/weights/best.pt')

# Run inference
model.predict('/content/datasets/Recognice-medications-boxes-5/test/images', save=True, imgsz=640, conf=0.2)


image 1/57 /content/datasets/Recognice-medications-boxes-5/test/images/008BE35A-ED85-43FA-BFD5-8344139510A8_jpeg.rf.36908adaf34f6631423c41b6f5fff7db.jpg: 640x640 1 BUSCAPINA, 10.0ms
image 2/57 /content/datasets/Recognice-medications-boxes-5/test/images/00BBDD2C-C2A1-463B-8D00-992EB02D6EC2_jpeg.rf.e0c4df7d27de86254dedac365ff1c7d4.jpg: 640x640 1 IBUPROFENO_GENFAR, 9.7ms
image 3/57 /content/datasets/Recognice-medications-boxes-5/test/images/01D0AD14-E522-4737-85C9-551BF4FF9D96_jpeg.rf.bff626e61ed25516b6b2de262afd7a19.jpg: 640x640 1 IBUFLASH_MIGRAN, 10.0ms
image 4/57 /content/datasets/Recognice-medications-boxes-5/test/images/067ACD18-F150-4040-AC28-5E64B7FA4799_jpeg.rf.77ba8b2ed0c9ccd79398fb07e45d2ca9.jpg: 640x640 1 ASPIRINA_100, 9.8ms
image 5/57 /content/datasets/Recognice-medications-boxes-5/test/images/0C7149D2-80E6-4D03-96D5-066D8308CA15_jpeg.rf.2f0eac5c8877981a5f5cf1c1d87511fb.jpg: 640x640 1 ALKASELTZER, 9.5ms
image 6/57 /content/datasets/Recognice-medications-boxes-5/test/images/0E

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'ALKASELTZER', 1: 'ASPIRINA_100', 2: 'BUSCAPINA', 3: 'DOLEX_GRIPA', 4: 'GAVISCON', 5: 'IBUFLASH_MIGRAN', 6: 'IBUPROFENO_GENFAR', 7: 'LEVONORGESTREL', 8: 'NORAVER_GRIPA_DIA', 9: 'VITAC_MK'}
 obb: None
 orig_img: array([[[ 97, 163,   4],
         [ 97, 163,   4],
         [ 97, 163,   4],
         ...,
         [103, 163,   3],
         [103, 163,   3],
         [103, 163,   3]],
 
        [[ 97, 163,   4],
         [ 97, 163,   4],
         [ 97, 163,   4],
         ...,
         [103, 163,   3],
         [103, 163,   3],
         [103, 163,   3]],
 
        [[ 97, 163,   4],
         [ 97, 163,   4],
         [ 97, 163,   4],
         ...,
         [103, 163,   3],
         [103, 163,   3],
         [103, 163,   3]],
 
        ...,
 
        [[ 13,  44,   5],
         [ 13,  44,   5],
         [ 13,  44,   5],
         ...,
         [  

In [35]:
#  Export model to tflite


from ultralytics import YOLO

# Load a model
model = YOLO('/content/datasets/runs/detect/train5/weights/best.pt')  # load a custom trained model

# Export the model
model.export(format='tflite')

Ultralytics YOLOv8.2.31 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 218 layers, 25845550 parameters, 0 gradients, 78.7 GFLOPs

PyTorch: starting from '/content/datasets/runs/detect/train5/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 14, 8400) (49.6 MB)

TensorFlow SavedModel: starting export with tensorflow 2.15.0...

ONNX: starting export with onnx 1.16.1 opset 17...
ONNX: slimming with onnxslim 0.1.28...
ONNX: export success ✅ 3.5s, saved as '/content/datasets/runs/detect/train5/weights/best.onnx' (98.9 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.22.3...
TensorFlow SavedModel: export success ✅ 110.7s, saved as '/content/datasets/runs/detect/train5/weights/best_saved_model' (247.4 MB)

TensorFlow Lite: starting export with tensorflow 2.15.0...
TensorFlow Lite: export success ✅ 0.0s, saved as '/content/datasets/runs/detect/train5/weights/best_saved_model/best_float32.tflite' (98.9 MB)

Export

'/content/datasets/runs/detect/train5/weights/best_saved_model/best_float32.tflite'

In [25]:
# Prediction using custom tflite model

#  Prediction using trained model

from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('/content/datasets/runs/detect/train4/weights/best_saved_model/best_float32.tflite')

# Run inference
model.predict('/content/datasets/Recognice-medications-boxes-5/test/images', save=True, imgsz=640, conf=0.2)

Loading /content/datasets/runs/detect/train3/weights/best_saved_model/best_float32.tflite for TensorFlow Lite inference...

image 1/57 /content/datasets/Recognice-medications-boxes-5/test/images/008BE35A-ED85-43FA-BFD5-8344139510A8_jpeg.rf.36908adaf34f6631423c41b6f5fff7db.jpg: 640x640 1 BUSCAPINA, 173.7ms
image 2/57 /content/datasets/Recognice-medications-boxes-5/test/images/00BBDD2C-C2A1-463B-8D00-992EB02D6EC2_jpeg.rf.e0c4df7d27de86254dedac365ff1c7d4.jpg: 640x640 1 IBUPROFENO_GENFAR, 152.5ms
image 3/57 /content/datasets/Recognice-medications-boxes-5/test/images/01D0AD14-E522-4737-85C9-551BF4FF9D96_jpeg.rf.bff626e61ed25516b6b2de262afd7a19.jpg: 640x640 1 IBUFLASH_MIGRAN, 155.7ms
image 4/57 /content/datasets/Recognice-medications-boxes-5/test/images/067ACD18-F150-4040-AC28-5E64B7FA4799_jpeg.rf.77ba8b2ed0c9ccd79398fb07e45d2ca9.jpg: 640x640 1 ASPIRINA_100, 153.3ms
image 5/57 /content/datasets/Recognice-medications-boxes-5/test/images/0C7149D2-80E6-4D03-96D5-066D8308CA15_jpeg.rf.2f0eac5c887

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'ALKASELTZER', 1: 'ASPIRINA_100', 2: 'BUSCAPINA', 3: 'DOLEX_GRIPA', 4: 'GAVISCON', 5: 'IBUFLASH_MIGRAN', 6: 'IBUPROFENO_GENFAR', 7: 'LEVONORGESTREL', 8: 'NORAVER_GRIPA_DIA', 9: 'VITAC_MK'}
 obb: None
 orig_img: array([[[ 97, 163,   4],
         [ 97, 163,   4],
         [ 97, 163,   4],
         ...,
         [103, 163,   3],
         [103, 163,   3],
         [103, 163,   3]],
 
        [[ 97, 163,   4],
         [ 97, 163,   4],
         [ 97, 163,   4],
         ...,
         [103, 163,   3],
         [103, 163,   3],
         [103, 163,   3]],
 
        [[ 97, 163,   4],
         [ 97, 163,   4],
         [ 97, 163,   4],
         ...,
         [103, 163,   3],
         [103, 163,   3],
         [103, 163,   3]],
 
        ...,
 
        [[ 13,  44,   5],
         [ 13,  44,   5],
         [ 13,  44,   5],
         ...,
         [  